In [1]:
# from datetime import datetime, timedelta
from googleads import adwords
# import codecs
# import time
# import os
# import csv
# import psycopg2   # For connection to postgres DB
import pandas as pd
# from sqlalchemy import create_engine
# import keyring
#import cStringIO

In [2]:
def get_accounts(client):
    managed_customer_service = client.GetService('ManagedCustomerService', version='v201710')
    selector = {
                'fields': ['Name','CustomerId'],
                'predicates': [
                    {
                        'field': 'Name',
                        'operator': 'DOES_NOT_CONTAIN_IGNORE_CASE',
                        'values': 'MCC'
                    },
                    {
                        'field': 'Name',
                        'operator': 'CONTAINS',
                        'values': 'Consumer | SEM | English'
                    },
                    {
                        'field': 'Name',
                        'operator': 'DOES_NOT_CONTAIN',
                        'values': 'Business'
                    }
                ]
    }
    accounts = managed_customer_service.get(selector)
    return accounts

columns = ['AccountDescriptiveName','AdGroupId', 'CampaignName','AdGroupName', 'Query', 'Impressions', 'Clicks', 'Conversions', 'Cost']
def get_data(client):
    accounts = get_accounts(client)
    report_downloader = client.GetReportDownloader(version='v201710')
    try:
        with open('sqr_raw.csv','wb') as a:
            for entry in accounts['links']:
                client.client_customer_id= entry['clientCustomerId']
                report = {
                      'reportName': 'SEARCH_QUERY_PERFORMANCE',
                      'dateRangeType': 'LAST_30_DAYS',
                      #'dateRangeType': 'CUSTOM_DATE',
                      'reportType': 'SEARCH_QUERY_PERFORMANCE_REPORT',
                      'downloadFormat': 'CSV',
                      'selector': {
                          #'dateRange' : {'min': 20170601, 'max': 20180412},
                          'fields': columns,
                          'predicates': [
                              {
                                  'field': 'Impressions',
                                  'operator': 'GREATER_THAN',
                                  'values': '4'
                              },
                              {
                                  'field': 'QueryTargetingStatus',
                                  'operator': 'EQUALS',
                                  'values': 'NONE'
                              },
                              {
                                  'field': 'CampaignName',
                                  'operator': 'DOES_NOT_CONTAIN',
                                  'values': 'DSA'
                              }
                          ]
                       },

                }
                report_downloader.DownloadReport(report,a)

    except Exception:
        print('error getting report')


##FILE IS PROCESSED
def remove_row (r):
    try:
        if r in ('Total', 'Account'):
            return 'yes'
        elif 'SEARCH_QUERY_' in r:
            return 'yes'
        else:
            return 'no'
    except Exception:
        print(r)
        print('error removing row')

In [3]:
if __name__ == '__main__':    # Check that this whole script is actually being run directly (as the main script) Then run the below
    adwords_client = adwords.AdWordsClient.LoadFromStorage('../googleads.yaml')
    get_data(adwords_client)

Your default encoding, cp1252, is not UTF-8. Please run this script with UTF-8 encoding to avoid errors.


In [4]:
dictionary = pd.read_excel('dictionary_relevance.xlsx')

## NLP

In [5]:
import string

In [6]:
from nltk.corpus import stopwords #might not be needed

In [7]:
def text_process(query):
    """
    removes all punctuation and returns a list of all words
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in query if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)

    return [word for word in nopunc.split()]

# VECTORIZATION

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(dictionary['Query'])

In [10]:
query_bow = bow_transformer.transform(dictionary['Query'])

# TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

In [12]:
tfidf_transformer = TfidfTransformer().fit(query_bow)

In [13]:
query_tfidf = tfidf_transformer.transform(query_bow)

# TRAINING A MODEL

In [14]:
from sklearn.naive_bayes import MultinomialNB

In [15]:
category_model = MultinomialNB().fit(query_tfidf, dictionary['Relevance'])

# MODEL EVALUATION

In [16]:
all_predictions = category_model.predict(query_tfidf)

In [17]:
from sklearn.metrics import classification_report

In [18]:
print (classification_report(dictionary['Relevance'], all_predictions))

             precision    recall  f1-score   support

       High       0.86      0.99      0.92      1206
        Low       0.97      0.94      0.96       898
     Medium       0.89      0.54      0.67       399

avg / total       0.90      0.90      0.89      2503



# TESTING

In [19]:
columns = ['AccountDescriptiveName','AdGroupId', 'CampaignName','AdGroupName', 'Query', 'Impressions', 'Clicks', 'Conversions', 'Cost']
df = pd.read_csv('sqr_raw.csv')
df.reset_index(inplace=True)
df.columns = columns
df['remove'] = df['AccountDescriptiveName'].apply(lambda x: remove_row(x))
#removed unnecessary rows
df.drop(df[df['remove'] == 'yes'].index, inplace=True)
df['Cost'] = df['Cost'].apply(lambda x: float(x)/1000000)
df.drop(['remove'], axis=1, inplace=True)
#df['engine'] = 'google'

In [20]:
def categorise_all(q):
    bow_q = bow_transformer.transform([q])
    tfidf_q = tfidf_transformer.transform(bow_q)
    return category_model.predict(tfidf_q)[0]

In [21]:
df['Relevance'] = df['Query'].apply(categorise_all)

In [22]:
df[['AccountDescriptiveName', 'AdGroupId', 'Query', 'Relevance', 'Impressions', 'Clicks', 'Conversions', 'Cost']].to_excel('negatives.xlsx')